In [1]:
from sympy.integrals.quadrature import gauss_legendre, gauss_laguerre
import sympy

In [2]:
npts = 400
digits = 100
x_leg, w_leg = gauss_legendre(npts, digits)

In [3]:
x_lag, w_lag = gauss_laguerre(npts, digits)

In [4]:
w_lag_weighted = []
for xs, ws in zip(x_lag, w_lag):
    w_lag_weighted.append(ws * sympy.exp(xs))

In [5]:
header = """
#ifndef QUADRATURE_WEIGHTS_H
#define QUADRATURE_WEIGHTS_H

#include <array>

#include "real_type.H"

// Gauss-Legendre and Gauss-Laguerre quadrature nodes and weights for {} points
// computed with SymPy.  For the Gauss-Legendre, we only keep the second half,
// since they are symmetric.

"""

In [6]:
with open(f"quadrature_weights_{npts}.H", "w") as of:
    of.write(header.format(npts))
    
    of.write("// Gauss-Legendre quadrature nodes and weights\n\n")

    of.write(f"inline const std::array<real_t, {npts//2}> x_leg = {{\n")
    for n, xs in enumerate(x_leg):
        if n < len(x_leg)//2:
            of.write(f"{4*' '}// str_to_real_t(\"{xs}\"),\n")
        elif n < len(x_leg)-1:
            of.write(f"{8*' '}str_to_real_t(\"{xs}\"),\n")
        else:
            of.write(f"{8*' '}str_to_real_t(\"{xs}\")}};\n\n\n")
            
    of.write(f"inline const std::array<real_t, {npts//2}> w_leg = {{\n")    
    for n, ws in enumerate(w_leg):
        if n < len(x_leg)//2:
            of.write(f"{4*' '}// str_to_real_t(\"{ws}\"),\n")
        elif n < len(x_leg)-1:
            of.write(f"{8*' '}str_to_real_t(\"{ws}\"),\n")
        else:
            of.write(f"{8*' '}str_to_real_t(\"{ws}\")}};\n\n")

    of.write("// Gauss-Laguerre quadrature nodes and weights.\n")
    of.write("// Note: the weights include the exp(x) factor already.\n")
    
    of.write(f"inline const std::array<real_t, {npts}> x_lag = {{\n")
    for n, xs in enumerate(x_lag):
        if n < len(x_leg)-1:
            of.write(f"{8*' '}str_to_real_t(\"{xs}\"),\n")
        else:
            of.write(f"{8*' '}str_to_real_t(\"{xs}\")}};\n\n")

    of.write(f"inline const std::array<real_t, {npts}> w_lag = {{\n")    
    for n, ws in enumerate(w_lag_weighted):
        if n < len(w_lag_weighted)-1:
            of.write(f"{8*' '}str_to_real_t(\"{ws}\"),\n")
        else:
            of.write(f"{8*' '}str_to_real_t(\"{ws}\")}};\n\n")
    
    of.write("#endif\n")